<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_with_DB_air_dialogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import tensorboard

In [ ]:
# Install Pytorch & other libraries
#!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


In [ ]:
import colab_env
import os

access_token = os.getenv("HUGGINGFACE_ACCESS_TOKEN")

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")


In [6]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [8]:
# set device
device = 'cuda'

In [9]:
torch.__version__

'2.3.0+cu121'

In [10]:
!apt-get update && apt-get install -y cuda-11-0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Fetched 3,733 kB in 4s (984 kB/s)
Reading package lists... Done

In [11]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Sun May 26 15:01:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   46C    P8       

In [12]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

In [2]:
def create_prompt_formats_air_dialogue(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"


In [3]:
from datasets import load_dataset
print("Preprocessing dataset air_dialogue")
dataset = load_dataset("google/air_dialogue")

dataset = dataset.map(create_prompt_formats_air_dialogue)

# save datasets to disk
dataset["train"].to_json("train_dataset_air_dialogue.json", orient="records")
dataset["validation"].to_json("test_dataset_air_dialogue.json", orient="records")

Preprocessing dataset air_dialogue


Map:   0%|          | 0/321459 [00:00<?, ? examples/s]

Map:   0%|          | 0/40363 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/322 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

65394255

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample'],
        num_rows: 321459
    })
    validation: Dataset({
        features: ['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample'],
        num_rows: 40363
    })
})


In [4]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="train_dataset_air_dialogue.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
print(dataset)

Dataset({
    features: ['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample'],
    num_rows: 321459
})


In [7]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [19]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            62Gi       2.0Gi        41Gi       2.0Mi        18Gi        60Gi
Swap:             0B          0B          0B


In [20]:
!git lfs install

Git LFS initialized.


In [21]:
!pip install git-lfs

In [22]:
!git lfs install
!git lfs pull

Git LFS initialized.
Not in a git repository.


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "nvidia/OpenMath-Mistral-7B-v0.1-hf" ###original

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    #attn_implementation="flash_attention_2", ## BertLMHeadModel does not support Flash Attention 2.0 yet
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    is_decoder=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
model, tokenizer = setup_chat_format(model, tokenizer)

In [25]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [26]:
text = "What is the capital of Canada?"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model.device)

generated_ids = model.generate(**model_inputs, temperature=0.9, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=128, do_sample=False)
decoded = tokenizer.decode(generated_ids[0])

print(decoded)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


<s> What is the capital of Canada?
Ottawa. The<|im_start|>-eyed<|im_start|><|im_start|><|im_start|> Ottawan,<|im_end|>


In [27]:
print(model_inputs)

{'input_ids': tensor([[    1,  1824,   349,   272,  5565,   302,  6082, 28804]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [28]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [29]:
index=10
prompt = dataset[index]
prompt

{'action': {'status': 'book', 'name': 'Amy Mitchell', 'flight': [1017]},
 'intent': {'return_month': 'Jan',
  'return_day': '26',
  'max_price': 5000,
  'departure_airport': 'IAH',
  'max_connections': 1,
  'departure_day': '24',
  'goal': 'book',
  'departure_month': 'Jan',
  'name': 'Amy Mitchell',
  'return_airport': 'BOS'},
 'timestamps': [1523245451,
  1523245460,
  1523245505,
  1523245514,
  1523245529,
  1523245547,
  1523245579,
  1523245582,
  1523245593,
  1523245665,
  1523245679,
  1523245681,
  1523245696,
  1523245700],
 'dialogue': ['customer: Hello.',
  'agent: Hello, How can I aid you?',
  'customer: I am Amy Mitchell want to travel to MA- BOS to enjoy beach holidays with my cousins.',
  'agent: Ok.',
  'customer:  Can you book a ticket from TX-IAH?',
  'agent: Sure, I am here to help you. May I know your travelling dates?',
  'customer: My travelling dates are 01/24 and 01/26.',
  'agent: Please wait.',
  'customer: Ok.',
  'agent: Thanks for waiting. You have a sing

In [30]:
index=10
prompt = dataset[index]['dialogue']
print(prompt)

prompt = dataset[index]['correct_sample']
prompt

['customer: Hello.', 'agent: Hello, How can I aid you?', 'customer: I am Amy Mitchell want to travel to MA- BOS to enjoy beach holidays with my cousins.', 'agent: Ok.', 'customer:  Can you book a ticket from TX-IAH?', 'agent: Sure, I am here to help you. May I know your travelling dates?', 'customer: My travelling dates are 01/24 and 01/26.', 'agent: Please wait.', 'customer: Ok.', 'agent: Thanks for waiting. You have a single connecting flight in JetBlue airline and flight number 1017 having price 100. Shall I proceed?', 'customer: Ok, you can proceed.', 'agent: Your ticket has been booked.', 'customer:  Thank you for booking.', 'agent: Thank you.']


True

MODEL GENERATION - ZERO SHOT

In [31]:
index=10

#features: ['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample'],
prompt = dataset[index]['dialogue']
summary = dataset[index]['correct_sample']

original_model = model

formatted_prompt = f"Instruct: summarize the following dialog.\n{prompt}\nOutput:\n"
res = model.generate(tokenizer.encode(formatted_prompt, return_tensors="pt"), max_length=512, do_sample=False)
output = tokenizer.decode(res[0], skip_special_tokens=True)


dash_line = '-'.join('' for x in range(100))
print(dash_line)
#print(f'INPUT PROMPT:\n{formatted_prompt}') # for dataset_dialogsum_test
print(f'QUESTION :\n{formatted_prompt}') # for dataset_squad2

print(dash_line)
#print(f'BASELINE HUMAN SUMMARY:\n{summary}\n') # for dataset_dialogsum_test
print(f'ANSWER BY HUMAN:\n{summary}\n') # for dataset_squad2
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}') # for dataset_dialogsum_test AND dataset_squad2

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
QUESTION :
Instruct: summarize the following dialog.
['customer: Hello.', 'agent: Hello, How can I aid you?', 'customer: I am Amy Mitchell want to travel to MA- BOS to enjoy beach holidays with my cousins.', 'agent: Ok.', 'customer:  Can you book a ticket from TX-IAH?', 'agent: Sure, I am here to help you. May I know your travelling dates?', 'customer: My travelling dates are 01/24 and 01/26.', 'agent: Please wait.', 'customer: Ok.', 'agent: Thanks for waiting. You have a single connecting flight in JetBlue airline and flight number 1017 having price 100. Shall I proceed?', 'customer: Ok, you can proceed.', 'agent: Your ticket has been booked.', 'customer:  Thank you for booking.', 'agent: Thank you.']
Output:

---------------------------------------------------------------------------------------------------
ANSWER BY HUMAN:
True

--------------------------------------------------------

In [32]:
#print(len(dataset_dialogsum_test))

print(len(dataset))

321459


In [33]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [8]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [9]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="OpenMath-Mistral-7B-v0.1-hf-dataset_air-dialogue", # 25/05/2024 #dataset_squad2

    #num_train_epochs=3,                     # number of training epochs
    num_train_epochs=1,                     # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

https://huggingface.co/docs/trl/en/sft_trainer

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_text_to_SQL.ipynb

In [12]:
print(dataset.column_names)

['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample']


In [10]:
text_field='Bob is a Traveller'

dataset = dataset.add_column('text', [text_field for example in dataset])

In [11]:
print(dataset.column_names)

['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample', 'text']


In [12]:
print(dataset[0]['text'])

Bob is a Traveller


In [15]:
torch.cuda.current_device()

0

In [13]:
from trl import SFTTrainer

import trl

# explicitly import SFTTrainer
#from trl.trainer import SFTTrainer


max_seq_length = 3072 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    args=args,


    train_dataset=dataset,

    #train_dataset=dataset_dialogsum_test,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        ##"per_device_train_batch_size": 8,
        #"per_device_eval_batch_size": 8,
        #"num_train_epochs": 3.0},
    }
)

Map:   0%|          | 0/321459 [00:00<?, ? examples/s]

In [ ]:
#del device
#model.to(torch.device('cuda'))
trainer.train()

# save model
trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


MIOS
cuda
cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,9.210300
20,7.740800
30,1.677500
40,4.486900
50,2.377000
60,1.260300
70,0.842400
80,0.436700
90,0.450200
100,0.294400


In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

Test Model and run Inference

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
#peft_model_id = "./Mistral-7B-text-to-sql"
#/content/Llama-2-7b-hf-text-to-sql-flash-attention-2
#peft_model_id = "./Llama-2-7b-hf-text-to-sql-flash-attention-2"
#peft_model_id = "./Llama-2-7b-hf-dialogsum-test-flash-attention-2"

#peft_model_id = "./OpenMath-Mistral-7B-v0.1-hf-dialogsum-test-flash-attention-2"

#peft_model_id = "./OpenMath-Mistral-7B-v0.1-hf-squad2-flash-attention-2"

peft_model_id = "./OpenMath-Mistral-7B-v0.1-hf-dataset_air-dialogue"

#peft_model_id = "./Mixtral-8x7B-Instruct-v0.1-text-to-sql"
#peft_model_id = "./Smaug-72B-v0.1-text-to-sql" # 04/03/2024
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline

#KeyError: "Unknown task text-generationn, available tasks are ['audio-classification',
#'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering',
# 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction',
#'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation',
#'ner', 'object-detection', 'question-answering', 'sentiment-analysis',
#'summarization', 'table-question-answering', 'text-classification', 'text-generation',
#'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification',
#'translation', 'video-classification', 'visual-question-answering', 'vqa',
#'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification',
#'zero-shot-object-detection', 'translation_XX_to_YY']"

#Upon looking at the HuggingFacePipeline source code, I found it only supports below pipelines:
#text-generation`, text2text-generation and summarization

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
print(model)

In [ ]:
print(model_id)

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen_after_tunning(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

MODEL GENERATION - AFTER TUNNING

In [ ]:
TUNE_model = model
index=10

formatted_prompt = f"Instruct: summarize the following dialog.\n{prompt}\nOutput:\n"

#features: ['action', 'intent', 'timestamps', 'dialogue', 'expected_action', 'search_info', 'correct_sample'],
prompt = dataset[index]['dialogue']
summary = dataset[index]['expected_action']

formatted_prompt = f"Instruct: summarize the following dialog.\n{prompt}\nOutput:\n"
#formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n" # for dataset_dialogsum_test
#formatted_prompt = f"Instruct: Answer the following question.\n{prompt}\nOutput:\n" # for dataset_squad2

res = gen_after_tunning(TUNE_model,formatted_prompt,1024,)
#print(res[0])
output = res[0].split('Output:\n')[1]


dash_line = '-'.join('' for x in range(100))
print(dash_line)
#print(f'INPUT PROMPT:\n{formatted_prompt}') # for dataset_dialogsum_test
print(f'QUESTION :\n{formatted_prompt}') # for dataset_squad2

print(dash_line)
#print(f'BASELINE HUMAN SUMMARY:\n{summary}\n') # for dataset_dialogsum_test
print(f'ANSWER BY HUMAN:\n{summary}\n') # for dataset_squad2
print(dash_line)
print(f'MODEL GENERATION - AFTER THE TUNNING:\n{output}') # for dataset_dialogsum_test AND dataset_squad2
